In [1]:
# (Optional) Enable interactive plotting in Jupyter

# For Jupyter notebooks
# %matplotlib notebook

# For Jupyter lab
%matplotlib widget

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

cwd = os.getcwd()
pardir = os.path.abspath(os.path.join(cwd, ".."))
modeldir = os.path.abspath(os.path.join(pardir, "models"))

if pardir not in sys.path:
    sys.path.append(pardir)

from jDAS import JDAS

# Goal of this tutorial

The basic denoising tutorial (`basic_denoising_example.ipynb`) shows how to perform basic denoising with `jDAS`. The example data used in that tutorial are obtained during the same DAS experiment as the data that was used in the training of the model, and so no retraining is needed. In this tutorial, we are going to retrain the model to obtain better accuracy on "new" data. In practice, retraining the model is recommended in the following scenarios:

1. A new experiment was conducted (e.g. in a different location, or with different interrogator settings like the gauge length or maximum sensing distance).
2. The conditions during one experiment strongly vary. This can be the case when new noise sources are introduced (construction works on-land, microseismic noise, etc.), or when the signal-to-noise ratio significantly changes.
3. One particular event of interest, such as a major earthquake, occurs. In this case the model can be trained in "single-sample mode": instead of training on a large data set and optimising the model parameters for a (potentially) wide data range, the training is done on a very specific data range. Consequently, the _jDAS_ model will try to achieve the best denoising performance for this specific data set, at the cost of generalisation.

Note that multiple models can be trained for different conditions (e.g. nighttime/daytime, on-land and submarine segments of the cable, etc.).

# Generate new data

As a simple example, we will generate some synthetic data. After corrupting these synthetics with noise (and bandpass filtering in the desired frequency band), we subsequently apply a standard pretrained model, retrain the model on new data, and apply the newly retrained model to see the difference in performance.

The synthetics are generated as a superposition of sines and cosines, filtered in space to introduce some spatio-temporal waveform coherence. These synthetic data will look very different from the original DAS data, which highlights the point of retraining the model.